<a href="https://colab.research.google.com/github/donghuna/AI-Expert/blob/main/2_1_Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


   # Autoencoders

## Overview

이번 실습은 autoencoder를 설계하고, 실제로 학습해보는 실습입니다.

시작하기에 앞서, 실습에 필요한 packages를 import 하겠습니다.

In [ ]:
import torch
import torch.nn as nn

import numpy as np

from tqdm import tqdm
from torchvision.utils import save_image, make_grid

DEVICE = torch.device("cuda")

## Fashion MNIST 데이터셋 로드

Fashion MNIST 데이터셋은 28x28 크기의 그레이 스케일 이미지로 구성된 데이터셋입니다.  
총 10개의 클래스를 합쳐 60,000개의 train set, 10,000개의 test set으로 구성되어있습니다.  
이번 실습은 Fashion MNIST 이미지 데이터를 latent vector로 압축하는 encoder와, 다시 이미지로 복구하는 decoder를 구현해보는 실습입니다.  

실습에 앞서 필요한 데이터를 로드하도록 하겠습니다.
아래 코드는 Fashion MNIST 데이터를 로드하는 코드입니다.

In [ ]:
from torchvision.datasets import FashionMNIST
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

dataset_path = '~/datasets'
batch_size = 100
test_batch_size = 1000

fashionmnist_transform = transforms.Compose([
        transforms.ToTensor(),
])

kwargs = {'num_workers': 1, 'pin_memory': True}

train_dataset = FashionMNIST(dataset_path, transform=fashionmnist_transform, train=True, download=True)
test_dataset  = FashionMNIST(dataset_path, transform=fashionmnist_transform, train=False, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
test_loader  = DataLoader(dataset=test_dataset,  batch_size=test_batch_size, shuffle=False, **kwargs)

## Basic autoencoder

다음 코드는 Fashion MNIST 데이터를 압축/복원하는 auto encoder 코드입니다.

### Implementation: auto encoder 구현
* input / latent / output  
  * input shape: batch x 28 x 28  
  * latent shape: batch x 64  
  * output shape: batch x 28 x 28  
* call  
  * input $x$ → latent $z$ → output $\hat{x}$

Example:
* Encoder  
  reshape: [batch, 1, 28, 28] → [batch, 784]  
  fc: [batch, 784] → [batch, latent_dim(64)]  

* Decoder  
  fc: [batch, latent_dim(64)] → [batch, 784]  
  reshape: [batch, 784] → [batch, 1, 28, 28] \\
  sigmoid: value_range: (-∞,∞) → (0, 1)    



In [ ]:
input_dim = 784
height = 28
width = 28
channel = 1

class AutoEncoder(nn.Module):
  def __init__(self, latent_dim=64):
    super().__init__()

    ############### ToDo ###############
    self.encoder = nn.Sequential(

    )
    self.decoder = nn.Sequential(

    )
    #####################################

  def encode(self, x):
    x = x.reshape((-1, input_dim))
    z = self.encoder(x)
    return z

  def decode(self, z):
    x_hat = self.decoder(z)
    x_hat = x_hat.reshape((-1, channel, height, width))
    return x_hat

  def forward(self, x):
    z = self.encode(x)
    x_hat = self.decode(z)
    return x_hat, z

In [ ]:
latent_dim = 64
model = AutoEncoder(latent_dim).to(DEVICE)

다음 코드는 auto encoder를 학습하기 위한 코드입니다.

In [ ]:
from torch.optim import Adam

lr = 1e-2
optimizer = Adam(model.parameters(), lr=lr)

In [ ]:
epochs = 10

for epoch in range(epochs):
    overall_loss = 0
    for batch_idx, (x, y) in enumerate(train_loader):
        x = x.to(DEVICE)

        optimizer.zero_grad()

        x_hat, z = model(x)
        ############### ToDo ###############
        loss =
        ####################################

        overall_loss += loss.item()

        loss.backward()
        optimizer.step()

    print("\tEpoch", epoch + 1, "complete!", "\tAverage Loss: ", overall_loss / (batch_idx*batch_size))

print("Finish!!")

아래 코드는 학습된 auto encoder 모델로 reconstruction을 진행하는 코드입니다.

In [ ]:
with torch.no_grad():
    x_test, y_test = next(iter(test_loader))
    x_hat_test, z_test = model(x_test.to(DEVICE))
    x_hat_test = x_hat_test.cpu()
    z_test = z_test.cpu()

n = 10
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 4))
for i in range(n):
  # display original
  ax = plt.subplot(2, n, i + 1)
  plt.imshow(x_test[i].permute(1, 2, 0))
  plt.title("original")
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

  # display reconstruction
  ax = plt.subplot(2, n, i + 1 + n)
  plt.imshow(x_hat_test[i].permute(1, 2, 0))
  plt.title("reconstructed")
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
plt.show()

# T-SNE

고차원의 복잡한 데이터들을 직관적으로 이해하기 어렵기 때문에, 일반적으로 낮은 차원으로 변환하여 시각화를 하여 분석을 하게 됩니다.  
t-SNE(t-Distributed Stochastic Neighbor Embedding)는 대표적인 차원 축소(dimensionality reduction) 기법입니다.  
아래 코드는 t-SNE를 이용하여 auto encoder로 얻어낸 embedding들을 시각화하는 코드입니다.

In [ ]:
from sklearn.manifold import TSNE
import seaborn as sns
import pandas as pd

def plot_tsne(inputs, labels):
  n_samples = len(labels)
  tsne = TSNE(n_components=2, perplexity=10, n_iter=300)
  tsne_features = tsne.fit_transform(inputs[:n_samples].reshape(n_samples, -1))
  df = pd.DataFrame({
      'x': tsne_features[:,0],
      'y': tsne_features[:,1],
      'label': labels[:n_samples]
  })

  sns.scatterplot(data=df, x='x', y='y', hue='label', palette="deep")

def plot(inputs, labels):
  n_samples = len(labels)

  df = pd.DataFrame({
      'x': inputs[:,0],
      'y': inputs[:,1],
      'label': labels[:n_samples]
  })

  sns.scatterplot(data=df, x='x', y='y', hue='label', palette="deep")

In [ ]:
plot_tsne(z_test, y_test)

In [ ]:
plot_tsne(x_test, y_test)

In [ ]:
with torch.no_grad():
    z_random = torch.randn(test_batch_size, latent_dim).to(DEVICE)
    x_hat_gen = model.decode(z_random)
    x_hat_gen = x_hat_gen.cpu()


n = 10
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 4))
for i in range(n):
  ax = plt.subplot(2, n, i + 1)
  plt.imshow(x_hat_gen[i].permute(1, 2, 0))
  plt.title("original")
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)